In [127]:
import numpy as np # array operations
import cv2 #opencv library read img operations
import os # use directory & join path
from pathlib import Path # 去掉副檔名
import pandas as pd # 匯出 csv
import struct
import math

# data visualisation and manipulation
import matplotlib.pyplot as plt #show img and table

from skimage.metrics import mean_squared_error
from skimage.metrics import peak_signal_noise_ratio
from skimage.metrics import structural_similarity

In [128]:
getBin = lambda x: x > 0 and str(bin(x))[2:] or "-" + str(bin(x))[3:]
 
def floatToBinary64(value):
    val = struct.unpack('Q', struct.pack('d', value))[0]
    return getBin(val)
 
def binaryToFloat(value):
    hx = hex(int(value, 2))   
    return struct.unpack("d", struct.pack("q", int(hx, 16)))[0]

def dec_to_bin_csv():
    DATASRC = 'feature/dec/'
    # 抓目錄下所有圖檔檔名
    src_name = os.listdir(DATASRC)
    for i in range (len(src_name)):

        # 檔案完整路徑
        src_path = DATASRC + src_name[i]
        data = pd.read_csv(src_path,header=None)
        
        mean = [floatToBinary64(float(data[1][1])),floatToBinary64(float(data[2][1])),floatToBinary64(float(data[3][1]))]
        std = [floatToBinary64(float(data[1][2])),floatToBinary64(float(data[2][2])),floatToBinary64(float(data[3][2]))]
        weight = [floatToBinary64(float(data[1][3])),floatToBinary64(float(data[2][3])),floatToBinary64(float(data[3][3]))]
        
        MeanStd = [mean,std,weight]
        MeanStd_table = pd.DataFrame(MeanStd)
        col_name = ['red','green','blue']
        row_name = ['Mean','Standard Deviation','weight']
        MeanStd_table.columns = col_name
        MeanStd_table.index = row_name
        # print(MeanStd_table)

        # 匯出 answer table 成 csv 檔
        src_name_temp= Path(src_name[i]).stem 
        src_name_tempp = src_name_temp.rstrip('dec')
        MeanStd_table.to_csv('output/bin/'+ src_name_tempp + 'bin.csv')

In [129]:
def get_mean_and_std(typeStr, img_name, x, x_weight):
    x_mean, x_std = cv2.meanStdDev(x)
    x_mean = np.hstack(np.around(x_mean,2))
    x_std = np.hstack(np.around(x_std,2))
    
    # 印出平均與標準差
    if(typeStr == 's'):
        print("\n來源圖檔名: ", img_name)
    elif(typeStr == 't'):
        print("\n目標圖檔名: ", img_name)
    
    MeanStd = [x_mean,x_std,x_weight]
    MeanStd_table = pd.DataFrame(MeanStd)
    col_name = ['red','green','blue']
    row_name = ['Mean','Standard Deviation','weight']
    MeanStd_table.columns = col_name
    MeanStd_table.index = row_name
    print(MeanStd_table)

    # 匯出 answer table 成 csv 檔
    MeanStd_table.to_csv('feature/dec/'+ img_name + '_dec.csv')
    # 轉成 IEEE bin
    dec_to_bin_csv()
    
    return x_mean, x_std

In [130]:
def color_transfer(src_lab, tar_lab, src_name, tar_name, coltra_name, coltra_num, weight_r, weight_g, weight_b):
    
    x_weight=[weight_r, weight_g, weight_b]
    s_mean, s_std = get_mean_and_std('s', src_name, src_lab, x_weight)
    t_mean, t_std = get_mean_and_std('t', tar_name, tar_lab, x_weight)

    # height, width, channel = src_rgb.shape
    
    coltra_lab = src_lab

    for k in range(0,3):
        x = src_lab[:,:,k]
        x = ((x-s_mean[k])*(((x_weight[k]*t_std[k])+(1-x_weight[k])*s_std[k])/s_std[k]))+ x_weight[k]*t_mean[k]+(1-x_weight[k])*s_mean[k]

        # # round or +0.5
        # x = np.round(x)
        x = np.ceil(x)

        # boundary check 超過邊界拉回在邊界上
        x = np.clip(x, 0, 255)
        coltra_lab[:,:,k] = x
    
    # 存顏色轉換結果圖
    str_weight = '_' + str(weight_r) + '_' + str(weight_g) + '_' + str(weight_b)
    # coltra_bgr = cv2.cvtColor(coltra_lab,cv2.COLOR_Lab2BGR)
    coltra_rgb = lab2rgb(coltra_lab)
    print(coltra_rgb)
    # coltra_bgr = cv2.cvtColor(coltra_rgb,cv2.COLOR_RGB2BGR)
    # cv2.imwrite('result/'+ coltra_name + str_weight + '.png', coltra_bgr)
    

In [131]:
def rgb2lab(rgb):

    # normalized to range [0,1]
    rgb = rgb / 255.0
    # rgb2xyz
    # xyz_transfer_matrix=[[0.5141,0.3239,0.1604],
    #                      [0.2651,0.6702,0.0641],
    #                      [0.0241,0.1228,0.8444]]
    # xyz = np.matmul(xyz_transfer_matrix,rgb)
    # # xyz2lms
    # lms_transfer_matrix=[[0.3897,0.6890,-0.0787],
    #                      [-0.2298,1.1834,0.0464],
    #                      [0.0000,0.0000,1.0000]]
    # lms = np.matmul(lms_transfer_matrix,xyz)

    # rgb2lms
    height, width, channel = rgb.shape

    lms_transfer_matrix=[[0.3811,0.5783,0.0402],
                         [0.1967,0.7244,0.0782],
                         [0.0241,0.1228,0.8444]]
    lms=rgb
    for h in range(0,height):
        for w in range(0,width):   
            lms[h][w] = np.matmul(lms_transfer_matrix,rgb[h][w])

    
    # lms2lms if lms < 0 set to 0.001 avoid log0 error
    # lms = 0.001 if lms <= 0 else lms
    lms = np.clip(lms,0.001,255)
    lms = np.log10(lms)
    
    # lms2lab
    matrix_1=[[(1/math.sqrt(3)),0,0],
           [0,(1/math.sqrt(6)),0],
           [0,0,(math.sqrt(2))]]
    matrix_2=[[1,1,1],
           [1,1,-2],
           [1,-1,0]]
    lab_transfer_matrix = np.matmul(matrix_1,matrix_2)
    print(len(lab_transfer_matrix))
    print(len(lms))
    print(lms.shape)

    lab = lms
    for h in range(0,height):
        for w in range(0,width): 
              lab[h][w] = np.matmul(lab_transfer_matrix,lms[h][w])

    return lab

In [132]:
def lab2rgb(lab):
    # lab2lms
    matrix_1=[[1,1,1],
              [1,1,-1],
              [1,-2,0]]
    matrix_2=[[(math.sqrt(3)/3),0,0],
              [0,(math.sqrt(6)/6),0],
              [0,0,(math.sqrt(2)/2)]]
    lms_transfer_matrix = np.matmul(matrix_1,matrix_2)
    height, width, channel = lab.shape
    lms = lab
    for h in range(0,height):
        for w in range(0,width): 
            lms[h][w] = np.matmul(lms_transfer_matrix,lab[h][w])

    # lms2lms
    for k in range(0,3):
        x = lms[:,:,k]
        x = np.power(10,k)
        lms[:,:,k] = x


    # lms2rgb
    rgb_transfer_matrix = [[4.4679,-3.5873,0.1193],
                           [-1.2186,2.3809,-0.1624],
                           [0.0497,-0.2439,1.2045]]

    rgb = lms
    for h in range(0,height):
        for w in range(0,width): 
            rgb[h][w] = np.matmul(rgb_transfer_matrix,lms[h][w])

    # round or +0.5
    rgb = np.floor(rgb * 255 + 0.5)
    return rgb

In [133]:
if __name__ == '__main__':

    weight_r, weight_g, weight_b = input("Enter 3 channel weight_red , weight_green, weight_blue(0.0 ≤ w ≤ 1.0):").split()
    
    weight_r = float(weight_r)
    weight_g = float(weight_g)
    weight_b = float(weight_b)
    
    # print("weight is: " + weight_r, weight_g, weight_b)
    # weight_r, weight_g, weight_b = 0.49,0.54,0.62

    # 印出所有圖片
    DATASRC = 'source/'
    DATATRG = 'target/'

    # 抓目錄下所有圖檔檔名
    src_name = os.listdir(DATASRC)
    tar_name = os.listdir(DATATRG)

    rgb_transfer_matrix = [[4.4679,-3.5873,0.1193],
                           [-1.2186,2.3809,-0.1624],
                           [0.0497,-0.2439,1.2045]]

    # for i in range (len(src_name)):
    for i in range (1):

            
        # 圖片完整路徑
        src_path = DATASRC + src_name[i]
        tar_path = DATATRG + tar_name[i]
        
        # convert img to array 以彩色格式讀取(三維)
        src_bgr = cv2.imread(src_path ,cv2.IMREAD_COLOR)   
        tar_bgr = cv2.imread(tar_path ,cv2.IMREAD_COLOR)
        
        # normalize
        # src_bgr.astype("float32") / 255
        # tar_bgr.astype("float32") / 255

        # 原為 BGR 轉為 RGB
        src_rgb = cv2.cvtColor(src_bgr,cv2.COLOR_BGR2RGB)
        tar_rgb = cv2.cvtColor(tar_bgr,cv2.COLOR_BGR2RGB)
        # src_lab = cv2.cvtColor(src_bgr,cv2.COLOR_BGR2Lab)
        # tar_lab = cv2.cvtColor(tar_bgr,cv2.COLOR_BGR2Lab)
        # print(src_rgb.shape)
        # print(src_rgb[0][0])
        # print(src_rgb[0][0].shape)
        # print(len(src_rgb[0][0]))
        # print(len(rgb_transfer_matrix))
        # lms = np.matmul(rgb_transfer_matrix,src_rgb[0][0])

        # RGB to lab
        src_lab = rgb2lab(src_rgb)
        tar_lab = rgb2lab(tar_rgb)

        # 去掉副檔名
        src_name_temp= Path(src_path).stem 
        tar_name_temp= Path(tar_path).stem 
        tar_name_tempp = tar_name_temp.lstrip('0'+str(i+1))
        coltra_name = src_name_temp + tar_name_tempp
        
        # 做色彩轉換
        color_transfer(src_lab, tar_lab, src_name_temp, tar_name_temp, coltra_name, i+1, weight_r, weight_g, weight_b)


3
768
(768, 512, 3)
3
512
(512, 768, 3)

來源圖檔名:  01_kodim17
                     red  green  blue
Mean               -1.14   0.06  0.02
Standard Deviation  0.65   0.07  0.03
weight              1.00   1.00  1.00

目標圖檔名:  01_kodim23
                     red  green  blue
Mean               -0.79   0.14  0.02
Standard Deviation  0.37   0.14  0.06
weight              1.00   1.00  1.00
[[[-4966.  1619. 30105.]
  [-4966.  1619. 30105.]
  [-4966.  1619. 30105.]
  ...
  [-4966.  1619. 30105.]
  [-4966.  1619. 30105.]
  [-4966.  1619. 30105.]]

 [[-4966.  1619. 30105.]
  [-4966.  1619. 30105.]
  [-4966.  1619. 30105.]
  ...
  [-4966.  1619. 30105.]
  [-4966.  1619. 30105.]
  [-4966.  1619. 30105.]]

 [[-4966.  1619. 30105.]
  [-4966.  1619. 30105.]
  [-4966.  1619. 30105.]
  ...
  [-4966.  1619. 30105.]
  [-4966.  1619. 30105.]
  [-4966.  1619. 30105.]]

 ...

 [[-4966.  1619. 30105.]
  [-4966.  1619. 30105.]
  [-4966.  1619. 30105.]
  ...
  [-4966.  1619. 30105.]
  [-4966.  1619. 30105.]
  [-4